<a href="https://colab.research.google.com/github/ivpaul/Inventory-Allocation/blob/main/Pacers_Georgetown_Inventory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas

In [2]:
import pandas as pd
import re

def is_shoe(size):
    if pd.isna(size):
        return False  # Return 0 for NaN values

    try:
        # Attempt to convert the size to a float, which covers integers and floats
        float(size)
        return True  # Numeric values are considered shoes
    except ValueError:
        # Updated regular expression to match 'M8/W9', '8/W9', and 'M8W9'
        if re.match(r'(M?\d+(\.\d+)?/W\d+(\.\d+)?)|(M?\d+(\.\d+)?W\d+(\.\d+)?)', size.upper()):
            return True  # The updated patterns are considered shoes

    return False  # All other values are not considered shoes

def identify_category(title):
    if "Women's" in title:
        return "Women's"
    elif "Men's" in title:
        return "Men's"
    elif "Unisex" in title:
        return "Unisex"
    else:
        return "Unknown"

# Define the function to categorize shoes by type
def is_stable(title):
    stability_keywords = [
        'kayano', 'cloudflyer', 'gaviota', 'arahi', 'gt-2000', 'paradigm',
        'cloudrunner', 'adrenaline gts', 'glycerin gts', 'guide', '860v13', '860v14', 'vongo', 'inspire'
    ]
    return any(keyword.lower() in title.lower() for keyword in stability_keywords)

def is_trail(title):
    trail_keywords = [
        'trail more', 'hierro', 'cloudvista', 'cloudultra', 'peregrine', 'timp', 'superior'
    ]
    return any(keyword.lower() in title.lower() for keyword in trail_keywords)

In [3]:

# Load the CSV file into a pandas DataFrame
Inventory_df = pd.read_csv('/inventory_export_1 2.csv')

Georgetown_Inventory_df = Inventory_df[['Title', 'Option1 Value', 'Option3 Value', 'Pacers Georgetown']]

# Add a new column indicating if the item is a shoe
# 'Option1 Value' should consist of integer values
Georgetown_Inventory_df['Is_Shoe'] = Georgetown_Inventory_df['Option1 Value'].apply(is_shoe)



<ipython-input-3-40591cf1c173>:2: DtypeWarning: Columns (10,14,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  Inventory_df = pd.read_csv('/inventory_export_1 2.csv')
<ipython-input-3-40591cf1c173>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Georgetown_Inventory_df['Is_Shoe'] = Georgetown_Inventory_df['Option1 Value'].apply(is_shoe)


In [4]:
#Check for Hoka Transport X

# Example: Looking up a title "Women's ASICS GEL-Kayano 30 Lite-Show"
specific_title = "Unisex HOKA ONE ONE Transport X"

# Use .loc[] with a condition to find rows where the 'Title' column matches `specific_title`
specific_row = Georgetown_Inventory_df.loc[Georgetown_Inventory_df['Title'] == specific_title]

# Display the found rows
print(specific_row.head(5))

                                 Title Option1 Value Option3 Value  \
8477   Unisex HOKA ONE ONE Transport X       M13/W14           NaN   
10600  Unisex HOKA ONE ONE Transport X         M8/W9           NaN   
10601  Unisex HOKA ONE ONE Transport X    M9.5/W10.5           NaN   
10602  Unisex HOKA ONE ONE Transport X        M9/W10           NaN   
10603  Unisex HOKA ONE ONE Transport X   M10.5/W11.5           NaN   

      Pacers Georgetown  Is_Shoe  
8477                  0     True  
10600       not stocked     True  
10601       not stocked     True  
10602       not stocked     True  
10603       not stocked     True  


In [5]:
# Filter the DataFrame for rows where 'Is_Shoe' is True
Georgetown_shoes_df = Georgetown_Inventory_df[Georgetown_Inventory_df['Is_Shoe'] == True]
print(Georgetown_shoes_df)

                                    Title Option1 Value Option3 Value  \
0           Men's Saucony Endorphin Pro 4            10             D   
1       Women's Saucony Endorphin Speed 4           9.5             B   
2         Men's Saucony Endorphin Speed 4          10.5             D   
3           Men's Saucony Endorphin Pro 4          10.5             D   
4       Women's Saucony Endorphin Speed 4            10             B   
...                                   ...           ...           ...   
102395         Men's Adidas Ultraboost 21            12             D   
102396         Men's Adidas Ultraboost 21          12.5             D   
102397         Men's Adidas Ultraboost 21          12.5             D   
102398         Men's Adidas Ultraboost 21          12.5             D   
102399         Men's Adidas Ultraboost 21            13             D   

       Pacers Georgetown  Is_Shoe  
0            not stocked     True  
1            not stocked     True  
2            no

In [6]:
# Replace 'not stocked' with 0
Georgetown_shoes_df['Pacers Georgetown'] = Georgetown_shoes_df['Pacers Georgetown'].replace('not stocked', 0)

# Convert 'Pacers Georgetown' to numeric in case there are other non-numeric values
Georgetown_shoes_df['Pacers Georgetown'] = pd.to_numeric(Georgetown_shoes_df['Pacers Georgetown'], errors='coerce')

# Filter the DataFrame to include only rows with 'Pacers Georgetown' > 0
Georgetown_shoes_clean_df = Georgetown_shoes_df[Georgetown_shoes_df['Pacers Georgetown'] > 0]

# Display the filtered DataFrame
print(Georgetown_shoes_clean_df)


                                     Title Option1 Value Option3 Value  \
302                 Men's ASICS GT-2000 12            11             D   
968                 Men's ASICS GT-2000 12             8             D   
980                 Men's ASICS GT-2000 12            10             D   
1152     Women's Mizuno Wave Rider 27 Roxy            10             B   
1216     Women's Mizuno Wave Rider 27 Roxy           7.5             B   
...                                    ...           ...           ...   
55129                Men's On Cloudmonster          12.5             D   
55287                   Women's On Cloud 5           8.5             B   
59665                 Women's Altra Timp 4           8.5             B   
70997  Unisex OOFOS OOahh Sport Flex Slide        M11W13           NaN   
73106   Men's OOFOS Ooahh Sport Flex Slide             9             D   

       Pacers Georgetown  Is_Shoe  
302                    1     True  
968                    1     True  
980

<ipython-input-6-b611fa6f6227>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Georgetown_shoes_df['Pacers Georgetown'] = Georgetown_shoes_df['Pacers Georgetown'].replace('not stocked', 0)
<ipython-input-6-b611fa6f6227>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Georgetown_shoes_df['Pacers Georgetown'] = pd.to_numeric(Georgetown_shoes_df['Pacers Georgetown'], errors='coerce')


In [7]:
print(Georgetown_shoes_clean_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 302 to 73106
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Title              1456 non-null   object
 1   Option1 Value      1456 non-null   object
 2   Option3 Value      1443 non-null   object
 3   Pacers Georgetown  1456 non-null   int64 
 4   Is_Shoe            1456 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 58.3+ KB
None


In [8]:
Georgetown_shoes_clean_df = Georgetown_shoes_clean_df.drop(['Pacers Georgetown', 'Is_Shoe'], axis=1)
Georgetown_shoes_clean_df = Georgetown_shoes_clean_df.rename(columns={'Title': 'Model'})
Georgetown_shoes_clean_df = Georgetown_shoes_clean_df.rename(columns={'Option1 Value': 'Size'})
Georgetown_shoes_clean_df = Georgetown_shoes_clean_df.rename(columns={'Option3 Value': 'Width'})

print(Georgetown_shoes_clean_df.head(5))


                                  Model Size Width
302              Men's ASICS GT-2000 12   11     D
968              Men's ASICS GT-2000 12    8     D
980              Men's ASICS GT-2000 12   10     D
1152  Women's Mizuno Wave Rider 27 Roxy   10     B
1216  Women's Mizuno Wave Rider 27 Roxy  7.5     B


In [9]:
# Create 'Gender' column and call identify_category function based on 'Model' values
Georgetown_shoes_clean_df['Gender'] = Georgetown_shoes_clean_df['Model'].apply(identify_category)

# Display the updated DataFrame
print(Georgetown_shoes_clean_df)


                                     Model    Size Width   Gender
302                 Men's ASICS GT-2000 12      11     D    Men's
968                 Men's ASICS GT-2000 12       8     D    Men's
980                 Men's ASICS GT-2000 12      10     D    Men's
1152     Women's Mizuno Wave Rider 27 Roxy      10     B  Women's
1216     Women's Mizuno Wave Rider 27 Roxy     7.5     B  Women's
...                                    ...     ...   ...      ...
55129                Men's On Cloudmonster    12.5     D    Men's
55287                   Women's On Cloud 5     8.5     B  Women's
59665                 Women's Altra Timp 4     8.5     B  Women's
70997  Unisex OOFOS OOahh Sport Flex Slide  M11W13   NaN   Unisex
73106   Men's OOFOS Ooahh Sport Flex Slide       9     D    Men's

[1456 rows x 4 columns]


In [10]:
# Assuming 'Georgetown_shoes_df' is your DataFrame and now includes a 'Gender_Category' column
gender_counts = Georgetown_shoes_clean_df['Gender'].value_counts()

# Print the breakdown
print("Breakdown by Gender:")
print(gender_counts)

# Total count
total_count = gender_counts.sum()
print(f"Total: {total_count}")


Breakdown by Gender:
Women's    723
Men's      721
Unisex      12
Name: Gender, dtype: int64
Total: 1456


In [11]:
# Define the function to categorize shoes by type

# Apply the function to create a new column for shoe type use function is_stable
Georgetown_shoes_clean_df['Stable'] = Georgetown_shoes_clean_df['Model'].apply(is_stable)

# Display the updated DataFrame to verify the new column

print(Georgetown_shoes_clean_df)

neutral_count = len(Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Stable'] == False])
stable_count = len(Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Stable'] == True])
print(f"Total Neutral shoes: {neutral_count}")
print(f"Total Stability shoes: {stable_count}")

                                     Model    Size Width   Gender  Stable
302                 Men's ASICS GT-2000 12      11     D    Men's    True
968                 Men's ASICS GT-2000 12       8     D    Men's    True
980                 Men's ASICS GT-2000 12      10     D    Men's    True
1152     Women's Mizuno Wave Rider 27 Roxy      10     B  Women's   False
1216     Women's Mizuno Wave Rider 27 Roxy     7.5     B  Women's   False
...                                    ...     ...   ...      ...     ...
55129                Men's On Cloudmonster    12.5     D    Men's   False
55287                   Women's On Cloud 5     8.5     B  Women's   False
59665                 Women's Altra Timp 4     8.5     B  Women's   False
70997  Unisex OOFOS OOahh Sport Flex Slide  M11W13   NaN   Unisex   False
73106   Men's OOFOS Ooahh Sport Flex Slide       9     D    Men's   False

[1456 rows x 5 columns]
Total Neutral shoes: 1040
Total Stability shoes: 416


In [12]:
# Define the function to categorize shoes by Terrain

# Apply the function to create a new column for shoe type use function is_trail
Georgetown_shoes_clean_df['Trail'] = Georgetown_shoes_clean_df['Model'].apply(is_trail)

# Display the updated DataFrame to verify the new column
print(Georgetown_shoes_clean_df)

road_count = len(Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Trail'] == False])
trail_count = len(Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Trail'] == True])
print(f"Total Road shoes: {road_count}")
print(f"Total Trail shoes: {trail_count}")

                                     Model    Size Width   Gender  Stable  \
302                 Men's ASICS GT-2000 12      11     D    Men's    True   
968                 Men's ASICS GT-2000 12       8     D    Men's    True   
980                 Men's ASICS GT-2000 12      10     D    Men's    True   
1152     Women's Mizuno Wave Rider 27 Roxy      10     B  Women's   False   
1216     Women's Mizuno Wave Rider 27 Roxy     7.5     B  Women's   False   
...                                    ...     ...   ...      ...     ...   
55129                Men's On Cloudmonster    12.5     D    Men's   False   
55287                   Women's On Cloud 5     8.5     B  Women's   False   
59665                 Women's Altra Timp 4     8.5     B  Women's   False   
70997  Unisex OOFOS OOahh Sport Flex Slide  M11W13   NaN   Unisex   False   
73106   Men's OOFOS Ooahh Sport Flex Slide       9     D    Men's   False   

       Trail  
302    False  
968    False  
980    False  
1152   False  


In [13]:
'''Print all unique shoe models in inventory'''

unique_models = Georgetown_shoes_clean_df['Model'].unique()

for title in unique_models:
    print(title)

Men's ASICS GT-2000 12
Women's Mizuno Wave Rider 27 Roxy
Women's ASICS Novablast 4
Men's ASICS Novablast 4
Women's New Balance FuelCell SuperComp Elite v4
Women's HOKA ONE ONE Clifton 9
Women's HOKA ONE ONE Bondi 8
Women's HOKA ONE ONE Arahi 7
Men's HOKA ONE ONE Bondi 8
Women's New Balance Fresh Foam X 880v14
Women's New Balance Fresh Foam X 1080v13
Women's New Balance Fresh Foam X Vongo v6
Women's On Cloudultra
Men's New Balance Fresh Foam X 880v14
Women's On Cloudgo
Women's On Cloudsurfer
Men's On Cloudsurfer
Women's On Cloudflow 4
Women's On Cloudeclipse
Men's On Cloudflow 4
Men's On Cloudultra 2
Men's On Cloudeclipse
Men's New Balance Fresh Foam X 1080v13
Men's On Cloudgo
Women's Altra Timp 5
Men's Altra Rivera 4
Women's Altra Rivera 4
Men's Altra Timp 5
Women's Brooks Glycerin 21
Women's Brooks Glycerin GTS 21
Men's Brooks Glycerin GTS 21
Men's Brooks Glycerin 21
Men's Saucony Guide 17
Men's Saucony Peregrine 14
Women's Saucony Guide 17
Women's Saucony Peregrine 14
Women's On Clou

In [14]:
'''Print all Stable shoe models'''

# Filter the DataFrame for rows where 'Stable' is True
stable_shoes_df = Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Stable'] == True]

# Get the unique values in the 'Model' column of the filtered DataFrame
unique_stable_models = stable_shoes_df['Model'].unique()

# Print the unique models
print("Unique Stable Shoe Models:")
for model in unique_stable_models:
    print(model)

Unique Stable Shoe Models:
Men's ASICS GT-2000 12
Women's HOKA ONE ONE Arahi 7
Women's New Balance Fresh Foam X Vongo v6
Women's Brooks Glycerin GTS 21
Men's Brooks Glycerin GTS 21
Men's Saucony Guide 17
Women's Saucony Guide 17
Women's Mizuno Wave Inspire 20
Men's Mizuno Wave Inspire 20
Men's HOKA ONE ONE Gaviota 5
Men's HOKA ONE ONE Arahi 7
Men's New Balance Fresh Foam X 860v13
Women's New Balance Fresh Foam X 860v13
Women's ASICS GT-2000 12
Women's Brooks Adrenaline GTS 23
Men's Brooks Adrenaline GTS 23
Women's ASICS GEL-Kayano 30
Men's New Balance Fresh Foam X Vongo v6
Women's ASICS GEL-Kayano 30 Anniversary
Men's On Cloudrunner
Men's ASICS GEL-Kayano 30 Anniversary
Men's On Cloudflyer 4
Men's ASICS GEL-Kayano 30
Women's Altra Paradigm 7
Men's Altra Paradigm 7
Women's On Cloudrunner


In [15]:
'''Print all Neutral shoe models'''

# Filter the DataFrame for rows where 'Stable' is False
neutral_shoes_df = Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Stable'] == False]

# Get the unique values in the 'Model' column of the filtered DataFrame
unique_neutral_models = neutral_shoes_df['Model'].unique()

# Print the unique models
print("Unique Neutral Shoe Models:")
for model in unique_neutral_models:
    print(model)

Unique Neutral Shoe Models:
Women's Mizuno Wave Rider 27 Roxy
Women's ASICS Novablast 4
Men's ASICS Novablast 4
Women's New Balance FuelCell SuperComp Elite v4
Women's HOKA ONE ONE Clifton 9
Women's HOKA ONE ONE Bondi 8
Men's HOKA ONE ONE Bondi 8
Women's New Balance Fresh Foam X 880v14
Women's New Balance Fresh Foam X 1080v13
Women's On Cloudultra
Men's New Balance Fresh Foam X 880v14
Women's On Cloudgo
Women's On Cloudsurfer
Men's On Cloudsurfer
Women's On Cloudflow 4
Women's On Cloudeclipse
Men's On Cloudflow 4
Men's On Cloudultra 2
Men's On Cloudeclipse
Men's New Balance Fresh Foam X 1080v13
Men's On Cloudgo
Women's Altra Timp 5
Men's Altra Rivera 4
Women's Altra Rivera 4
Men's Altra Timp 5
Women's Brooks Glycerin 21
Men's Brooks Glycerin 21
Men's Saucony Peregrine 14
Women's Saucony Peregrine 14
Women's On Cloud 5 WP
Women's Saucony Ride 17
Men's HOKA ONE ONE Clifton 9
Women's HOKA ONE ONE Transport
Women's HOKA ONE ONE Mach X
Men's HOKA ONE ONE Transport
Men's HOKA ONE ONE Mach X


In [16]:
'''Print all Trail shoe models'''

# Filter the DataFrame for rows where 'Trail' is False
trail_shoes_df = Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Trail'] == True]

# Get the unique values in the 'Model' column of the filtered DataFrame
unique_trail_models = trail_shoes_df['Model'].unique()

# Print the unique models
print("Unique Trail Shoe Models:")
for model in unique_trail_models:
    print(model)

Unique Trail Shoe Models:
Women's On Cloudultra
Men's On Cloudultra 2
Women's Altra Timp 5
Men's Altra Timp 5
Men's Saucony Peregrine 14
Women's Saucony Peregrine 14
Women's On Cloudvista
Men's On Cloudvista
Women's New Balance Fresh Foam X Trail More v3
Men's New Balance Fresh Foam X Trail More v3
Women's On Cloudultra 2
Men's New Balance Fresh Foam X Hierro V7
Women's Altra Timp 4


In [17]:
print(Georgetown_shoes_clean_df.head(5))

                                  Model Size Width   Gender  Stable  Trail
302              Men's ASICS GT-2000 12   11     D    Men's    True  False
968              Men's ASICS GT-2000 12    8     D    Men's    True  False
980              Men's ASICS GT-2000 12   10     D    Men's    True  False
1152  Women's Mizuno Wave Rider 27 Roxy   10     B  Women's   False  False
1216  Women's Mizuno Wave Rider 27 Roxy  7.5     B  Women's   False  False


In [18]:
'''Women's Wide Shoes'''

# Filter for wide shoes based on gender and width conditions
womens_wide_shoes_df = Georgetown_shoes_clean_df[((Georgetown_shoes_clean_df['Gender'] == 'Women\'s') & (Georgetown_shoes_clean_df['Width'] == 'D'))]

# Print the filtered DataFrame
print("Women's Wide Shoes Inventory:")
print(womens_wide_shoes_df)

# Print the number of wide shoes for women
num_womens_wide_shoes = len(womens_wide_shoes_df)
print(f"\nNumber of Women's Wide Shoes: {num_womens_wide_shoes}")

# If you just want to print unique models of wide shoes
unique_women_wide_models = womens_wide_shoes_df['Model'].unique()
print("\nUnique Models of Wide Shoes:")
for model in unique_women_wide_models:
    print(model)


Women's Wide Shoes Inventory:
                                         Model  Size Width   Gender  Stable  \
1721            Women's HOKA ONE ONE Clifton 9     7     D  Women's   False   
3708   Women's New Balance Fresh Foam X 880v14   7.5     D  Women's   False   
3711   Women's New Balance Fresh Foam X 880v14     8     D  Women's   False   
3714   Women's New Balance Fresh Foam X 880v14   8.5     D  Women's   False   
3718   Women's New Balance Fresh Foam X 880v14     9     D  Women's   False   
...                                        ...   ...   ...      ...     ...   
41636                  Women's Brooks Ghost 15    10     D  Women's   False   
41637                  Women's Brooks Ghost 15  10.5     D  Women's   False   
41638                  Women's Brooks Ghost 15    11     D  Women's   False   
41666                  Women's Brooks Ghost 15   9.5     D  Women's   False   
41694                  Women's Brooks Ghost 15     7     D  Women's   False   

       Trail  
1721  

In [19]:
'''Men's Wide Shoes'''

# Filter for wide shoes based on gender and width conditions
mens_wide_shoes_df = Georgetown_shoes_clean_df[((Georgetown_shoes_clean_df['Gender'] == 'Men\'s') & (Georgetown_shoes_clean_df['Width'] == '2E'))]

# Print the filtered DataFrame
print("Men's Wide Shoes Inventory:")
print(mens_wide_shoes_df)

# Print the number of wide shoes for women
num_mens_wide_shoes = len(mens_wide_shoes_df)
print(f"\nNumber of Men's Wide Shoes: {num_mens_wide_shoes}")

# If you just want to print unique models of wide shoes
unique_men_wide_models = mens_wide_shoes_df['Model'].unique()
print("\nUnique Models of Wide Shoes:")
for model in unique_men_wide_models:
    print(model)


Men's Wide Shoes Inventory:
                                       Model  Size Width Gender  Stable  Trail
4353   Men's New Balance Fresh Foam X 880v14    12    2E  Men's   False  False
4359   Men's New Balance Fresh Foam X 880v14  12.5    2E  Men's   False  False
4364   Men's New Balance Fresh Foam X 880v14    13    2E  Men's   False  False
4418   Men's New Balance Fresh Foam X 880v14     9    2E  Men's   False  False
4424   Men's New Balance Fresh Foam X 880v14   9.5    2E  Men's   False  False
...                                      ...   ...   ...    ...     ...    ...
42167                  Men's Brooks Ghost 15  10.5    2E  Men's   False  False
42172                  Men's Brooks Ghost 15     9    2E  Men's   False  False
42175                  Men's Brooks Ghost 15    11    2E  Men's   False  False
42193                  Men's Brooks Ghost 15  12.5    2E  Men's   False  False
42194                  Men's Brooks Ghost 15  10.5    2E  Men's   False  False

[79 rows x 6 columns]



In [30]:
'''Total Report'''
print(f"\nTotal Men's Wide Shoes: {num_mens_wide_shoes}")
print(f"\nTotal Women's Wide Shoes: {num_womens_wide_shoes}")
print(f"\nTotal Road Shoes: {road_count}")
print(f"\nTotal Trail Shoes: {trail_count}")

# Correctly filter for unisex shoes
uni_shoes_df = Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Gender'] == 'Unisex']

# # If you want to print the entire DataFrame of unisex shoes
# print("Unisex Shoes Inventory:")
# print(uni_shoes_df)

# Print the number of unisex shoes
num_uni_shoes = len(uni_shoes_df)  # or uni_shoes_df.shape[0]
print(f"\nTotal Unisex Shoes: {num_uni_shoes}")


Total Men's Wide Shoes: 79

Total Women's Wide Shoes: 92

Total Road Shoes: 1371

Total Trail Shoes: 85

Total Unisex Shoes: 12


In [47]:
#Check for specific shoe

# Example: Looking up a title "Women's ASICS GEL-Kayano 30 Lite-Show"
# specific_title = Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Model'] == "AMZ-Women's Saucony Kinvara 14"]
# specific_title = Georgetown_shoes_clean_df[Georgetown_shoes_clean_df['Model'] == "Unisex HOKA ONE ONE Transport X"]
specific_title = Georgetown_shoes_clean_df[(Georgetown_shoes_clean_df['Model'] == "Men's HOKA ONE ONE Transport") |
                                               (Georgetown_shoes_clean_df['Model'] == "Women's HOKA ONE ONE Transport")]

# Print the filtered DataFrame
print(specific_title)

                                Model  Size Width   Gender  Stable  Trail
6801   Women's HOKA ONE ONE Transport   8.5     B  Women's   False  False
6819   Women's HOKA ONE ONE Transport   9.5     B  Women's   False  False
6833   Women's HOKA ONE ONE Transport    10     B  Women's   False  False
6868   Women's HOKA ONE ONE Transport     7     B  Women's   False  False
6879   Women's HOKA ONE ONE Transport   7.5     B  Women's   False  False
6901     Men's HOKA ONE ONE Transport    11     D    Men's   False  False
6910     Men's HOKA ONE ONE Transport    12     D    Men's   False  False
6981     Men's HOKA ONE ONE Transport     9     D    Men's   False  False
6987     Men's HOKA ONE ONE Transport   9.5     D    Men's   False  False
6992     Men's HOKA ONE ONE Transport    10     D    Men's   False  False
6995     Men's HOKA ONE ONE Transport  10.5     D    Men's   False  False
20199    Men's HOKA ONE ONE Transport  11.5     D    Men's   False  False
22225  Women's HOKA ONE ONE Transport 